## Dependencies

In [19]:
!pip install clean-text
!pip install inflect


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
datatype = 'dev' 

path = (
    'PIZZA_train.json' if datatype == 'train' else
    'PIZZA_dev.json' if datatype == 'dev' else
    'PIZZA_test.json' if datatype == 'test' else
    'unknown.json'  # Optional fallback
)

lines = (
    20 if datatype == 'train' else
    30 if datatype == 'dev' else
    1357 if datatype == 'test' else
    0  
)

print(f"Path: {path}")
print(f"Lines: {lines}")


Path: PIZZA_dev.json
Lines: 30


## Parsing JSON File

In [22]:

#! load data given file path and number of line to load
def load_data(file_path,num_lines):
    data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline())) 
    return data
def load_random_data(file_path, num_lines, last_lines):
    data = []
    
    with open(file_path, 'r') as file:
        file.seek(0, 2)  # Move to the end of the file to get its size
        file_size = file.tell()
        
        # Read the last `last_lines` lines
        file.seek(max(file_size - 1024 * last_lines, 0))  # Optional: Adjust the seek window size for efficiency
        lines = file.readlines()
        last_lines_data = [json.loads(line.strip()) for line in lines[-last_lines:] if line.strip()]
        data.extend(last_lines_data)

        # Read `num_lines` random lines from the rest of the file (excluding the last `last_lines` lines)
        remaining_lines = file_size - sum(len(line) for line in lines[-last_lines:])
        for _ in range(num_lines):
            while True:
                # Pick a random position outside the last `last_lines` range
                random_pos = random.randint(0, remaining_lines - 1)
                file.seek(random_pos)
                
                # Read to the end of the current line to avoid partial lines
                file.readline()
                line = file.readline()  # Read the next line (complete line)
                if line:  # Ensure it's not an empty line
                    # Check if it's not from the last `last_lines` lines
                    # if any(line.strip() == last_line for last_line in last_lines_data):
                    #     continue  # Skip if it's from the last lines
                    try:
                        data.append(json.loads(line.strip()))
                        break  # Exit the loop for this line
                    except json.JSONDecodeError:
                        continue  # Skip if it's not valid JSON
    return data


In [23]:
data=load_data(path,lines) if datatype=='train' else load_data(path,lines)
print(data[:3])

[{'dev.SRC': 'i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage', 'dev.EXR': '(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )', 'dev.TOP': '(ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )', 'dev.PCFG_ERR': 'False'}, {'dev.SRC': 'five medium pizzas with tomatoes and ham', 'dev.EXR': '(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE MED

In [24]:
#! split json data
def getextensions(datatype):
    if datatype=='train':
        return 'train.SRC','train.EXR','train.TOP'
    elif datatype=='dev':
        return 'dev.SRC','dev.EXR','dev.TOP'
    else:
        return 'test.SRC','test.EXR','test.TOP'
def get_training_data(data,datatype='train'):
    values=getextensions(datatype)
    training_data = []
    training_exr=[]
    training_top=[]
    # training_top_dec=[]
    for item in data:
        training_data.append(item[values[0]])  
        training_exr.append(item[values[1]])  
        training_top.append(item[values[2]])
        # if datatype=='train':
        #     training_top_dec.append(item[values[3]])  
    return training_data,training_exr,training_top
training_data,training_exr,training_top=get_training_data(data,datatype)   
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])


['i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage', 'five medium pizzas with tomatoes and ham']
['(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )', '(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE MEDIUM ) (TOPPING HAM ) (TOPPING TOMATOES ) ) )']
['(ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )', '(ORDER (PIZZAORDER (NUMBER five )

In [25]:
# Create an inflect engine
p = inflect.engine()

def replace_numbers_with_words(text):
    # Regular expression to find numbers in the text
    number_pattern = r'\b\d+\b'
    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')

    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        return p.number_to_words(number, andword="").replace('-', '_')
    
    # Substitute numbers with their word representation
    text = re.sub(number_pattern, number_to_words, text)
    
    # Replace the standalone 'a' with 'one' only when it reprsents a
    text = re.sub(
        r'\b[aA]\b(?! ((lot of)|(little)|(few)|(number of)|(couple of)|(number of)|(bit of)|(number of)|(bit)|(load of)|(stack of)|(bunch of)|(group of)|(set of)|(series of)|(variety of)|(range of)|(number of)|(amount of)|(sum of)|(total)))',
        'one',
        text
    )
    return text

In [26]:
#! a function to handle negations
def handle_negations(text):
    #! negations are based on training set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [27]:
#!# lemmatize words with all possible pos tags
def lem_word(word):
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [28]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords.update(not_stopwords)
# stopwords.discard('a')
# stopwords.discard('an')
# stopwords.discard('not')
# stopwords.discard('no')
# stopwords.discard('can')
# stopwords.discard('not_a')
# stopwords.discard('not_an')
# stopwords.discard('not_can')
# stopwords.discard('not_no')
stopwords=set()
# stopwords.add('and')
# stopwords.add('also')

In [29]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    # training_data = [handle_negations(order) for order in training_data]
    training_data=[replace_numbers_with_words(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # print(training_data)
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    # print(training_data)
    # print(training_data)
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    #! remove "can" at the beginning of the sentence
    # training_data=[re.sub(r'^can\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # training_data = [[lemmatizer.lemmatize(word) for word in order] for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
# print(training_data[:5])

In [30]:
#! save processed training dataset
path = (
    'trian_data_order_category.txt' if datatype == 'train' else
    'dev_data_order_category.txt'
)
with open(path, 'a') as f: #### dev_data_processed.txt
    for item in training_data:
        f.write("%s\n" % item)

In [31]:
#! a utility function for extra parentheses ) removal 
#! handles COMPLEX_TOPPING, NOT,... parenthesis cases
def remove_unmatched_parentheses(input_string):
    result = list(input_string)  # Convert to list for mutability
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
        elif char == ')' and i+2 >= len(result):
            result[i] = ''
            result[last_bracket_index] = ')'
    return ''.join(result)

In [32]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    # print(training_data)
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order=replace_numbers_with_words(order)
            order = re.sub(r"[\(\)]", "", order)
            order=word_tokenize(order) #! fix id and don't bugs
            print(order)
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            order=re.sub(r'\bd\s+', '', order) #! for d removal
            print(order)
            tokens = word_tokenize(order)
            print(tokens)
            # tokens = [lemmatizer.lemmatize(word) for word in tokens]
            tokens = [lem_word(word) for word in tokens]
            print(tokens)
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)

['PIZZAORDER', 'two', 'medium', 'pizzas', 'with', 'sausage', 'and', 'black', 'olives']
pizzaorder two medium pizzas with sausage and black olives
['pizzaorder', 'two', 'medium', 'pizzas', 'with', 'sausage', 'and', 'black', 'olives']
['pizzaorder', 'two', 'medium', 'pizza', 'with', 'sausage', 'and', 'black', 'olive']
['PIZZAORDER', 'two', 'medium', 'pizzas', 'with', 'pepperoni', 'and', 'extra', 'cheese']
pizzaorder two medium pizzas with pepperoni and extra cheese
['pizzaorder', 'two', 'medium', 'pizzas', 'with', 'pepperoni', 'and', 'extra', 'cheese']
['pizzaorder', 'two', 'medium', 'pizza', 'with', 'pepperoni', 'and', 'extra', 'cheese']
['PIZZAORDER', 'three', 'large', 'pizzas', 'with', 'pepperoni', 'and', 'sausage']
pizzaorder three large pizzas with pepperoni and sausage
['pizzaorder', 'three', 'large', 'pizzas', 'with', 'pepperoni', 'and', 'sausage']
['pizzaorder', 'three', 'large', 'pizza', 'with', 'pepperoni', 'and', 'sausage']
['PIZZAORDER', 'five', 'medium', 'pizzas', 'with', 't

In [33]:
path = (
    'train_labels_order_category.txt' if datatype == 'train' else
    'dev_labels_order_category.txt'
)
for labels in order_category_labels:
    with open(path, 'a') as f: # dev
        f.write("%s\n" % labels)

In [34]:
def remove_unmatched_parentheses(text):
    stack = []
    result = list(text)
    for i, char in enumerate(text):
        if char == '(':
            stack.append(i)
        elif char == ')':
            if stack:
                stack.pop()
            else:
                result[i] = ''  # Remove unmatched closing parenthesis
    for i in stack:  # Remove unmatched opening parentheses
        result[i] = ''
    return ''.join(result)

def process_training_top(training_top, stopwords):
    labeled_data = []
    for i, item in enumerate(training_top):
        sentence_tokens = []
        labels = []
        # Remove unwanted keywords like PIZZAORDER or DRINKORDER
        unwanted_keywords = r'\(\b(ORDER|PIZZAORDER|DRINKORDER)\b'
        cleaned_string = re.sub(unwanted_keywords, "", item)
        # Remove stopwords and unnecessary parentheses
        words = cleaned_string.split()
        cleaned_words = [word for word in words if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_words)

        # Remove unmatched parentheses
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        # Regex to match nested parentheses excluding COMPLEX_TOPPING
        pattern_regex = r"\((?!COMPLEX_TOPPING\b)(\w+)\s+((?:[^\(\)]|\([^()]*\))*)\)"
        matches = list(re.finditer(pattern_regex, cleaned_string))
        for _, match in enumerate(matches):
            key, value = match.groups()
            # Handle NOT block
            if key == "NOT":
                nested_match = re.match(pattern_regex, value.strip())
                if nested_match:
                    nested_key, nested_value = nested_match.groups()
                    value_tokens = word_tokenize(nested_value.strip())
                    sentence_tokens.extend(value_tokens)
                    labels.extend([f"NOT_{nested_key}"] * len(value_tokens))
                continue

            value = re.sub(r'\(\s*\w+\s*', '', value)  # Match '(WORD)' for complex toppings
            value = re.sub(r'[()]', '', value)  # Remove any remaining parentheses

            value_tokens = replace_numbers_with_words(value.strip())
            value_tokens = clean(value_tokens, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            value_tokens = re.sub(r'\bd\s+', '', value_tokens)  # Remove 'd'
            value_tokens = word_tokenize(value_tokens)
            value_tokens = [lem_word(word) for word in value_tokens]
            sentence_tokens.extend(value_tokens)

            # Labeling logic
            label_mapping = {
                "STYLE": "STYLE",
                "SIZE": "SIZE",
                "TOPPING": "Topping",
                "NUMBER": "Number",
                "QUANTITY": "Complex-Topping",
                "DRINKTYPE": "Drinktype",
                "CONTAINERTYPE": "ContainerType",
            }
            labels.extend([label_mapping.get(key, "None")] * len(value_tokens))
        if sentence_tokens:
            labeled_data.append((sentence_tokens, labels))

    return labeled_data

def update_training_labels(training_data, training_labels, labeled_data):
    # Define a mapping for new labels to start from 3
    label_mapping = {
        "Number": 3,
        "SIZE": 4,
        "STYLE": 5,
        "Topping": 6,
        "Complex-Topping": 7,
        "NOT_TOPPING": 8,
        "Drinktype": 9,
        "ContainerType": 10,
        "None": 11
    }

    new_training_data = [list(tokens) for tokens in training_data]
    new_training_labels = [list(labels) for labels in training_labels]

    for idx, (train_tokens, train_labels) in enumerate(zip(new_training_data, new_training_labels)):
        for labeled_tokens, labeled_labels in labeled_data:
            if set(labeled_tokens).issubset(set(train_tokens)):
                token_to_label = {
                    token: label_mapping.get(label, label_mapping["None"]) for token, label in zip(labeled_tokens, labeled_labels)
                }
                for i, token in enumerate(train_tokens):
                    if token in token_to_label:
                        train_labels[i] = token_to_label[token]

    return new_training_data, new_training_labels

labeled_data = process_training_top(training_top, stopwords)
Model2_data, Model2_labels = update_training_labels(training_data, order_category_labels, labeled_data)
print(training_data[8:10])
print(order_category_labels[8:10]) 
print(Model2_data[8:10])
print(Model2_labels[8:10])


[['i', 'want', 'to', 'have', 'five', 'pie', 'with', 'pepper', 'pesto', 'and', 'onion', 'please'], ['can', 'i', 'get', 'two', 'pie', 'with', 'pepper', 'and', 'bacon', 'pesto']]
[[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0]]
[['i', 'want', 'to', 'have', 'five', 'pie', 'with', 'pepper', 'pesto', 'and', 'onion', 'please'], ['can', 'i', 'get', 'two', 'pie', 'with', 'pepper', 'and', 'bacon', 'pesto']]
[[2, 2, 2, 2, 3, 0, 0, 6, 6, 0, 6, 2], [2, 2, 2, 3, 0, 0, 6, 0, 6, 6]]


In [35]:

path = (
    'train_data_order_details.txt' if datatype == 'train' else
    'dev_data_order_details.txt'
)

with open(path, 'a') as f: 
    for item in Model2_data:
        f.write("%s\n" % item)

In [36]:
path = (
    'train_labels_order_details.txt' if datatype == 'train' else
    'dev_labels_order_details.txt'
)
for labels in Model2_labels:
    with open(path, 'a') as f:
        f.write("%s\n" % labels)